In [14]:
# !pip3 install datasets
# ! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [1]:
!cp "/content/drive/MyDrive/imp docs/ML_Review data.csv" /content/.

Read data

In [2]:
import pandas as pd

review_data_raw = pd.read_csv("ML_Review data.csv")

In [3]:
review_data_raw

,ProductID,Review,battery life,camera quality,screen,performance,price,Rating
0,P001,The battery life is good. The camera quality i...,1,1,0,1,0,4
1,P001,"Overall, this product is excellent.",0,0,0,0,0,5
2,P001,The battery life is disappointing. The screen ...,1,0,1,1,0,2
3,P001,The screen is satisfactory. The performance is...,0,0,1,1,1,4
4,P001,The screen is terrible. The price is the worst...,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...
9995,P020,The battery life is perfect. The camera qualit...,1,1,1,0,0,5
9996,P020,The screen is okay. I had higher expectations.,0,0,1,0,0,3
9997,P020,The camera quality is outstanding. The screen ...,0,1,1,1,0,5
9998,P020,The battery life is bad. The camera quality is...,1,1,0,1,0,2


Cleaning the review

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


ps = PorterStemmer()

stop_words = set(stopwords.words('english'))


def clean_text(sentence):

    tokens = [ps.stem(x) for x in word_tokenize(sentence)]
    tokens = [x  for x in tokens if x not in stop_words]
    return " ".join(tokens)

review_data_raw['clean_review'] = review_data_raw['Review'].apply(clean_text)

In [6]:
review_data_raw['clean_review']

0       batteri life good . camera qualiti quit nice ....
1                            overal , thi product excel .
2       batteri life disappoint . screen disappoint . ...
3       screen satisfactori . perform quit nice . pric...
4          screen terribl . price worst . higher expect .
                              ...                        
9995    batteri life perfect . camera qualiti perfect ...
9996                        screen okay . higher expect .
9997    camera qualiti outstand . screen excel . perfo...
9998    batteri life bad . camera qualiti bad . perfor...
9999    batteri life satisfactori . camera qualiti sat...
Name: clean_review, Length: 10000, dtype: object

In [7]:
def get_sentiment(rating):
    if rating>=4:
        return 0 # "Postive"
    elif rating == 3:
        return 1 #"Neutral"
    else:
        return 2 #"Negative"
review_data_raw['sentiment'] = review_data_raw['Rating'].apply(get_sentiment)

In [8]:
review_data_raw['sentiment'].value_counts()

0    5053
1    2496
2    2451
Name: sentiment, dtype: int64

Training a distil bert model for sentiment analysis using pytorch

In [9]:
# train test split
from sklearn.model_selection import train_test_split
train_reviews , test_reviews = train_test_split(review_data_raw, test_size=0.33, random_state=42 , stratify=review_data_raw['sentiment'])

Made split in equal percentage

In [10]:
print(train_reviews['sentiment'].value_counts(normalize=True))
print(test_reviews['sentiment'].value_counts(normalize=True))

0    0.505373
1    0.249552
2    0.245075
Name: sentiment, dtype: float64
0    0.505152
1    0.249697
2    0.245152
Name: sentiment, dtype: float64


For distilbert text preprocessing is not required so we can go ahead with the orignal reviews

We will finetune distill bert model for sentiment classification

In [11]:
from datasets import Dataset
train_reviews = train_reviews[['Review' , 'sentiment']]
test_reviews = test_reviews[['Review' ]]
train_reviews = train_reviews.rename(columns={"Review": "text", "sentiment": "label"})
test_reviews = test_reviews.rename(columns={"Review": "text", "sentiment": "label"})
train_reviews = Dataset.from_pandas(train_reviews)
test_reviews = Dataset.from_pandas(test_reviews)

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=512)

tokenized_train_review = train_reviews.map(preprocess_function, batched=True)
tokenized_test_review = test_reviews.map(preprocess_function, batched=True)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

In [13]:
# tokenized_train_review.data.table['input_ids'].size

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# !pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [21]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_review,
    eval_dataset=tokenized_test_review,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TypeError: Trainer.predict() missing 1 required positional argument: 'test_dataset'

In [24]:
results = trainer.evaluate()

In [32]:
results = trainer.predict(tokenized_test_review)

In [38]:
predictions = results.predictions.argmax(axis=1)

In [42]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report


In [44]:
print(classification_report(test_reviews['label'] , predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1667
           1       1.00      1.00      1.00       824
           2       1.00      1.00      1.00       809

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300



## correlation PART 2

using tfidf to find the key sentiment indecators

In [ ]:
review_data_raw = pd.read_csv("ML_Review data.csv")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(review_data_raw['Review'])
vectorizer.get_feature_names_out()